In [35]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import os
from scipy import stats as s

In [2]:
# open the .txt file which have names of training videos
f = open("trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [3]:
# open the .txt file which have names of test videos
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head(100)

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi
...,...
95,Archery/v_Archery_g04_c01.avi
96,Archery/v_Archery_g04_c02.avi
97,Archery/v_Archery_g04_c03.avi
98,Archery/v_Archery_g04_c04.avi


In [4]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [5]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    #print(videoFile)
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if frameId%math.floor(frameRate) == 0:
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg"%count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 9537/9537 [01:58<00:00, 80.35it/s] 


In [6]:
# getting the names of all the images
images = glob("train_1/*.jpg")
print(len(images))
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[1])
    # creating the class of image
    train_class.append(images[i].split('/')[1].split('_')[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('UCF/train_new.csv',header=True, index=False)

100%|██████████| 10645/10645 [00:00<00:00, 522044.36it/s]

10645


In [7]:
train = pd.read_csv('UCF/train_new.csv')
train.head()

,image,class
0,v_Billiards_g24_c02.avi_frame4.jpg,Billiards
1,v_Billiards_g21_c06.avi_frame7.jpg,Billiards
2,v_Biking_g23_c01.avi_frame14.jpg,Biking
3,v_BlowingCandles_g17_c03.avi_frame5.jpg,BlowingCandles
4,v_BodyWeightSquats_g15_c04.avi_frame3.jpg,BodyWeightSquats


In [8]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 10645/10645 [00:37<00:00, 286.75it/s]


(10645, 224, 224, 3)

In [9]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)


In [10]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [11]:
print(X_test.shape)
print(X_train.shape)

(2129, 224, 224, 3)
(8516, 224, 224, 3)


In [12]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [13]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8516, 7, 7, 512)

In [14]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2129, 7, 7, 512)

In [15]:
# reshaping the training as well as validation frames in single dimension
X_test = X_test.reshape(2129, 7*7*512)
X_train = X_train.reshape(8516, 7*7*512)

In [16]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [17]:
# shape of images
X_train.shape

(8516, 25088)

In [18]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='softmax'))

In [19]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')


In [20]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [21]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(8516, 25088)
(8516, 17)
(2129, 25088)
(2129, 17)


In [22]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
67/67 [==============================] - 18s 272ms/step - loss: 2.3976 - accuracy: 0.2295 - val_loss: 1.7693 - val_accuracy: 0.3776
Epoch 2/200
67/67 [==============================] - 21s 320ms/step - loss: 1.5791 - accuracy: 0.4617 - val_loss: 0.8701 - val_accuracy: 0.7529
Epoch 3/200
67/67 [==============================] - 18s 272ms/step - loss: 0.8561 - accuracy: 0.7125 - val_loss: 0.3303 - val_accuracy: 0.9220
Epoch 4/200
67/67 [==============================] - 19s 282ms/step - loss: 0.4371 - accuracy: 0.8562 - val_loss: 0.1339 - val_accuracy: 0.9667
Epoch 5/200
67/67 [==============================] - 16s 232ms/step - loss: 0.2591 - accuracy: 0.9209 - val_loss: 0.0811 - val_accuracy: 0.9765
Epoch 6/200
67/67 [==============================] - 16s 241ms/step - loss: 0.1786 - accuracy: 0.9462 - val_loss: 0.0645 - val_accuracy: 0.9822
Epoch 7/200
67/67 [==============================] - 15s 226ms/step - loss: 0.1330 - accuracy: 0.9602 - val_loss: 0.0715 - val_accuracy:

67/67 [==============================] - 12s 181ms/step - loss: 0.0299 - accuracy: 0.9931 - val_loss: 0.0506 - val_accuracy: 0.9930
Epoch 58/200
67/67 [==============================] - 12s 183ms/step - loss: 0.0226 - accuracy: 0.9939 - val_loss: 0.0395 - val_accuracy: 0.9930
Epoch 59/200
67/67 [==============================] - 13s 189ms/step - loss: 0.0240 - accuracy: 0.9940 - val_loss: 0.0252 - val_accuracy: 0.9948
Epoch 60/200
67/67 [==============================] - 12s 181ms/step - loss: 0.0236 - accuracy: 0.9932 - val_loss: 0.0553 - val_accuracy: 0.9915
Epoch 61/200
67/67 [==============================] - 12s 183ms/step - loss: 0.0379 - accuracy: 0.9914 - val_loss: 0.0602 - val_accuracy: 0.9906
Epoch 62/200
67/67 [==============================] - 13s 187ms/step - loss: 0.0296 - accuracy: 0.9924 - val_loss: 0.0539 - val_accuracy: 0.9901
Epoch 63/200
67/67 [==============================] - 12s 182ms/step - loss: 0.0212 - accuracy: 0.9947 - val_loss: 0.0565 - val_accuracy: 0.992

67/67 [==============================] - 13s 199ms/step - loss: 0.0224 - accuracy: 0.9939 - val_loss: 0.0251 - val_accuracy: 0.9958
Epoch 114/200
67/67 [==============================] - 12s 183ms/step - loss: 0.0227 - accuracy: 0.9941 - val_loss: 0.0312 - val_accuracy: 0.9962
Epoch 115/200
67/67 [==============================] - 13s 189ms/step - loss: 0.0184 - accuracy: 0.9951 - val_loss: 0.0303 - val_accuracy: 0.9948
Epoch 116/200
67/67 [==============================] - 12s 178ms/step - loss: 0.0142 - accuracy: 0.9961 - val_loss: 0.0317 - val_accuracy: 0.9953
Epoch 117/200
67/67 [==============================] - 12s 175ms/step - loss: 0.0311 - accuracy: 0.9926 - val_loss: 0.0278 - val_accuracy: 0.9958
Epoch 118/200
67/67 [==============================] - 12s 177ms/step - loss: 0.0221 - accuracy: 0.9951 - val_loss: 0.0329 - val_accuracy: 0.9939
Epoch 119/200
67/67 [==============================] - 12s 177ms/step - loss: 0.0258 - accuracy: 0.9952 - val_loss: 0.0438 - val_accuracy:

67/67 [==============================] - 13s 188ms/step - loss: 0.0152 - accuracy: 0.9968 - val_loss: 0.0435 - val_accuracy: 0.9953
Epoch 170/200
67/67 [==============================] - 13s 190ms/step - loss: 0.0101 - accuracy: 0.9973 - val_loss: 0.0400 - val_accuracy: 0.9934
Epoch 171/200
67/67 [==============================] - 13s 188ms/step - loss: 0.0154 - accuracy: 0.9967 - val_loss: 0.0293 - val_accuracy: 0.9953
Epoch 172/200
67/67 [==============================] - 13s 195ms/step - loss: 0.0151 - accuracy: 0.9977 - val_loss: 0.0465 - val_accuracy: 0.9934
Epoch 173/200
67/67 [==============================] - 13s 193ms/step - loss: 0.0161 - accuracy: 0.9958 - val_loss: 0.0514 - val_accuracy: 0.9934
Epoch 174/200
67/67 [==============================] - 14s 209ms/step - loss: 0.0211 - accuracy: 0.9953 - val_loss: 0.0502 - val_accuracy: 0.9925
Epoch 175/200
67/67 [==============================] - 15s 223ms/step - loss: 0.0216 - accuracy: 0.9946 - val_loss: 0.0398 - val_accuracy:

In [24]:
base_model = VGG16(weights='imagenet', include_top=False)

In [25]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='softmax'))

In [29]:
# loading the trained weights
model.load_weights("weight.hdf5")

In [30]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [31]:
# getting the test list
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [32]:
# creating the tags
train = pd.read_csv('UCF/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [39]:
test_videos = test_videos[:659]

In [40]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg"%count
            count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])
    

100%|██████████| 659/659 [18:14<00:00,  1.66s/it]


In [41]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

73.14112291350531